In [184]:
import random
import nltk
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer 
from sklearn.svm import LinearSVC, SVC
from sklearn.linear_model import LogisticRegression
import codecs

In [185]:
BOT_CONFIG = {
    'intents': {
        'hello': {
            'examples': ['Hello', 'Good day', 'Hey', 'Wazzap', 'Yo'],
            'responses': ['Hello', "Welcome"]
                },
        'brands': {
            'examples': ['Brands', 'Can you shaw me available vehicle brands, please?', 'available vehicle brands'],
            'responses': ['Tesla, Nissan, NIO', 'Tesla, NIO, Nissan']
                },
        'cruise_range': {
            'examples': ['range', 'cruise range', 'Can you show me cars with the maximum cruising range'],
            'responses': ['Sure. That is Tesla Model S (640 km), NIO ec6 (615 km), NIO es6 (610 km)', 'Tesla Model S (640 km), NIO ec6 (615 km), NIO es6 (610 km)']
                },
        
        'bye': {
            'examples': ['Bye', 'Have a nice day', 'Good bye'], 
            'responses': ['If you need me  I will be here', 'See you', 'Bye', 'Have a nice day', 'Good bye' ]
        }
    },
        
    'failure_phrases': [
        'Please write it in another way',
        'Something is wrong',
        'I do not understand. Repfrase your request, please',
        'The list of my answers is limited. Ask the question in another way'
    ]
}

In [195]:
X_texts = []
y = []



for intent, intent_data in BOT_CONFIG['intents'].items():
    for example in intent_data['examples']:
        X_texts.append(example)
        y.append(intent)
            

vectorizer = TfidfVectorizer(analyzer='char_wb', ngram_range=(2, 4))
# vectorizer = TfidfVectorizer(analyzer='char_wb')
# vectorizer = CountVectorizer()
X = vectorizer.fit_transform(X_texts)
clf = LinearSVC().fit(X, y)

In [ ]:
def get_intent(question):
    question_vector = vectorizer.transform([question])
    intent = clf.predict(question_vector)[0]
    
    examples = BOT_CONFIG['intents'][intent]['examples'] 
    for example in examples:
        dist = nltk.edit_distance(question, example)
        dist_percentage = dist / len(example)
        if dist_percentage < 0.4:
            return intent
    
    
#     index = list(clf_proba.classes_).index(intent)
#     proba = clf_proba.predict_proba(question_vector)[0][index]
#     print(intent, proba)
#     if proba > 0.2:
#         return intent
    

In [197]:
def get_answer_by_intent(intent):
    if intent in BOT_CONFIG['intents']:
        phrases = BOT_CONFIG['intents'][intent]['responses']
        return random.choice (phrases)

In [206]:
def filter_text(text):
    text = text.lower()
    text = [c for c in text if c in 'abcdefghijklmnopqrstuvwxyz- ']
    text = ''.join(text)
    return text


# fileObj = codecs.open( "dia.txt", "r", "utf_8_sig" )
with open('dia.txt') as f:
    content = f.read()
    
dialogues = [dialogue_line.split('\n') for dialogue_line in content.split('\n\n')]

questions = set ()
qa_dataset = []

for replicas in dialogues:
    if len(replicas) < 2:
        continue
        
    question, answer = replicas[:2]
    question = filter_text(question[2:])
    answer = answer[2:]
    
    if question and question not in questions:
        questions.add(question)
        qa_dataset.append([question, answer])


qa_by_word_dataset = {} # {'word': [[q, a], ...]}
for question, answer in qa_dataset:
    words = question.split(' ')
    for word in words:
        if word not in qa_by_word_dataset:
            qa_by_word_dataset[word] = []
        qa_by_word_dataset[word].append((question, answer))


        
qa_by_word_dataset_filtered = {word: qa_list
                               for word, qa_list in qa_by_word_dataset.items()
                               if len(qa_list) < 1000}



def generate_answer_by_text(text):
    text = filter_text(text)
    words = text.split(' ')
    qa = []
    for word in words:
        if word in qa_by_word_dataset_filtered:
            qa += qa_by_word_dataset_filtered[word]
    qa = list(set(qa))[:1000]
        
    results =[]
    for question, answer in qa:
        dist = nltk.edit_distance(question, text)
        dist_percentage = dist / len(question)
        results.append([dist_percentage, question, answer])
        
    if results:
        dist_percentage, question, answer = min(results, key=lambda pairs: pairs[0])
        if dist_percentage < 0.2:
            return answer

[['I need to request a wake-up call for tomorrow morning.'], ['What time do you want the call?'], ['', 'I need two calls, one at 7 and another at 7:15.'], ['We can certainly do that. Expect a call from us at 7:00, and then again at 7:15.'], ['', 'Actually, can I change the latter wake-up call to 7:30 am?'], ['I can certainly do that. Is there anything else?'], ['', "I can't think of anything. If I do think of something, I'll be sure to call again."], ['Okay. Good night, sir.'], ['', 'I need a wake-up call tomorrow morning.'], ['Of course. When would you like the call?'], ['', 'Actually, I need two calls, one at 7 and the other at 7:15.'], ["No problem. We'll give you both calls."], ['', 'This bus goes all the way to Santa Anita mall, right?'], ["Yes, it'll take us there."], ['', 'Are you positive?'], ['I always catch this bus.'], ['', 'How long is this bus ride?'], ['It only takes half an hour.'], ['', 'Where do we get off at?'], ["We can get off the bus right behind Macy's."]]


In [207]:
def get_failure_phrase():
    phrases = BOT_CONFIG ['failure_phrases']
    return random.choice (phrases)

In [208]:
stats = [0, 0, 0]

In [209]:
def bot(question):
    #NLU
    intent = get_intent(question)
    
    #get answer
    
    #We are seaching ready answer
    if intent:
        answer = get_answer_by_intent(intent)
        if answer:
            stats[0] += 1
            return answer
        
    #Generate eligible answer by text
    answer = generate_answer_by_text(question)
    if answer:
        stats[1] += 1
        return answer
    
    #zaglushka
    stats[2] += 1
    answer = get_failure_phrase()
    return answer
    

In [ ]:
question = None

while question not in ['exit','finish']:
    question = input()
    answer = bot(question)
    print (answer, stats)

kjn
Something is wrong [0, 0, 1]
lkj
The list of my answers is limited. Ask the question in another way [0, 0, 2]
